In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 



In [15]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=False,
    fill_mode='nearest'
)

In [4]:
val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '../dataset/train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 254347 images belonging to 82 classes.


In [5]:
validation_generator = val_test_datagen.flow_from_directory(
    '../dataset/val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 63637 images belonging to 82 classes.


In [6]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=outputs)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=20
)


In [ ]:
for layer in base_model.layers[-30:]:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=10
)

In [10]:
model.save('handwritten_equation_model.h5')
test_generator = val_test_datagen.flow_from_directory(
    '../dataset/test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 79545 images belonging to 82 classes.


In [11]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")

2486/2486 ━━━━━━━━━━━━━━━━━━━━ 3506s 1s/step - accuracy: 0.8255 - loss: 0.6100
Test accuracy: 0.8233


In [12]:
predictions = model.predict(test_generator)
predicted_classes = predictions.argmax(axis=1)

2486/2486 ━━━━━━━━━━━━━━━━━━━━ 3203s 1s/step


In [14]:
from sklearn.metrics import classification_report 
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

               precision    recall  f1-score   support

            !       0.00      0.00      0.00       260
            (       0.04      0.05      0.05      2859
            )       0.04      0.05      0.04      2871
            +       0.06      0.07      0.07      5023
            ,       0.00      0.00      0.00       382
            -       0.08      0.08      0.08      6800
            0       0.02      0.02      0.02      1831
            1       0.08      0.07      0.08      5753
            2       0.07      0.07      0.07      5676
            3       0.03      0.05      0.04      2623
            4       0.02      0.02      0.02      1916
            5       0.02      0.01      0.01      1135
            6       0.02      0.01      0.02      1049
            7       0.00      0.00      0.00      1006
            8       0.01      0.02      0.02      1031
            9       0.01      0.02      0.01      1152
            =       0.04      0.04      0.04      2621
         